In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda 사용가능한지
print(f'Using {device} device')

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() # super함수: 자식 클래스에서 부모 클래스 사용
        self.flatten = nn.Flatten() # reshape를 모델 내에서 동작시켜주는 것, 행렬을 배열로 바꿈
        self.linear_relu_stack = nn.Sequential( # 계층을 선형으로 쌓는 모델?
            nn.Linear(28*28, 512),
            nn.ReLU(), # (x<0, 0, x>0, x)
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    # 입력데이터에 대한 연산 구현
    
    def forward(self, x): 
        x = self.flatten(x)
        logits = self.linear_relu_stack(x) # 
        return logits

In [6]:
model = NeuralNetwork().to(device) # 인공신경망의 인스턴스를 device로 이동
print(model) #구조 출력

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # 모델에 입력, 각 class에 대한 원시(raw) 예측값이 있는 10-차원 텐서 반환
pred_probab = nn.Softmax(dim=1)(logits) # sofmax는 분류에 쓰는 활성화 함수, 0부터 1사이의 실수로 출력, 확률로 표현가능
y_pred = pred_probab.argmax(1)
print(f"Predictaed class: {y_pred}")

Predictaed class: tensor([3])


In [10]:
input_image = torch.rand(3,28,28) # 28*28 크기의 이미지 3개
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten() # nn.Flatten 계층 초기화
flat_image = flatten(input_image) # 28X28의 2D 이미지를 784 픽셀 값의 연속된 배열로 변환
print(flat_image.size()) 

torch.Size([3, 784])


In [14]:
# weight와 bias를 이용해서 입력에 선형 변환을 적용

layer1 = nn.Linear(in_features=28*28, out_features=20) # 입력의 차원, 출력의 차원(28*28 의 입력에 20의 출력)
hidden1 = layer1(flat_image) # hidden에 layer1 배열 저장
print(hidden1.size())

torch.Size([3, 20])


In [15]:
# 비선형 활성화 (선형 변형 후 적용, nonlinearity 도입)
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[-0.2506,  0.5663,  0.0989,  0.0360, -0.3089, -0.0369, -0.0620,  0.2613,
         -0.0835, -0.5532,  0.7165, -0.2788,  0.3703,  0.4521, -0.3496, -0.0682,
         -0.0070,  0.2308, -0.0097,  0.0367],
        [-0.2743,  0.3305,  0.0526, -0.0312, -0.0675, -0.2042, -0.1554,  0.2646,
         -0.6164, -0.4548,  0.6736, -0.3767,  0.0754,  0.1100, -0.2118,  0.1009,
          0.2345,  0.1904, -0.1883,  0.2789],
        [-0.2841,  0.7093,  0.1378,  0.1426, -0.5160,  0.0334, -0.1894,  0.3681,
         -0.3065, -0.6643,  0.5256, -0.2598,  0.1597,  0.2538, -0.2637,  0.0342,
          0.5077,  0.5076, -0.1487, -0.1477]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.5663, 0.0989, 0.0360, 0.0000, 0.0000, 0.0000, 0.2613, 0.0000,
         0.0000, 0.7165, 0.0000, 0.3703, 0.4521, 0.0000, 0.0000, 0.0000, 0.2308,
         0.0000, 0.0367],
        [0.0000, 0.3305, 0.0526, 0.0000, 0.0000, 0.0000, 0.0000, 0.2646, 0.0000,
         0.0000, 0.6736, 0.0000, 0.0754, 0.1100, 0.00